# CLI

```
poodle [Files/Folders to mutate]
    --runner {runner name or path}
    --report {report name, repeatable}  # replaces default list of reporters
    --max_parallel -P {number max parallel runners}
    --base_folder_name {}
```

# Config 

## poodle_config.py
```
targets: list[str] = [File and Folder names to mutate]

runner = Runner Name str or Runner Object

# all mutators enabled by default ???
register_mutators: dict = {
    Name: object
} 
disable_mutators: list[str] = [Names to disable]
enable_mutators: list[str] = [Names to enable]

# only enable sysout by default ???
register_reporters = {
    Name: object
}
disable_reporters: list[str] = [Names to disable]???
enable_reporters: list[str] = [Names to enable]

max_parallel: int = default to system.affil cup?

base_folder: str = "poodle-runner"
```

# files
cli.py
* Responsible for CLI interface and calling core.
core.py
* Resolve configuration (defaults, poodle_config.py, toml, args)
* Init Mutators
* Find list of ".py" files to mutate
* Parse Py file, pass to each mutator, build list of PoodleMutant objects
* Create pool of Runners
* As each runner is available, call with PoodleMutant, aggregate results
* Print out current status every few seconds
* When all done, Init and call Reporter

# Mutator
* If class, will init with kwargs: poodle_config: dict
* create_mutants(module: ast.Module, poodle_config: dict) -> list or generator? [ast.Module or PoodleMutant?]

# Runner
* Must be class, will init with kwargs: folder: Path, poodle_config: dict
* test_mutation(mutant: PoodleMutant, timeout_s: int) -> (pass: bool, timeout: bool, reason: str | None)

# Reporter
* If class, will init with kwargs: poodle_config: dict
* create_report(report: PoodleReport, poodle_config: dict)


In [ ]:
import ast
from pathlib import Path
from pprint import pprint

file = "example/src/operators.py"

module = ast.parse(Path(file).read_bytes(), file)
# pprint(ast.dump(m,indent=4))

# {type(node) for node in ast.walk(module) if not hasattr(node,"lineno")}

for node in ast.walk(module):
    print(
        (
            type(node),
            node.lineno if hasattr(node,"lineno") else None,
            node.col_offset if hasattr(node,"col_offset") else None,
            node.end_lineno if hasattr(node,"end_lineno") else None,
            node.end_col_offset if hasattr(node,"end_col_offset") else None,
        )
    )

In [ ]:
import ast
from ast import Add, BinOp, Sub
from copy import deepcopy
from pprint import pprint

mod = ast.parse("a = 1 + 2 - 3")
# pprint(ast.dump(mod, indent=4))

mod_c = deepcopy(mod)
for node in ast.walk(deepcopy(mod_c)):
    if type(node) == BinOp and type(node.op) == Add:
        op = Sub
        node.op = op()
        print((ast.unparse(node), node.col_offset, node.end_col_offset, node.lineno, node.end_lineno))

# pprint(ast.dump(mod_c, indent=4))
# ast.unparse(mod_c)
# ast.unparse(mod)

In [ ]:
import os
os.getcwd().split(os.sep)[-1]

In [ ]:
os.getcwd().split(os.sep)

In [ ]:
from pathlib import Path

p = Path("src/poodle/core.py")



In [ ]:
from pathlib import Path
from pprint import pprint

# 2,11,2,16


file_text = Path("example/src/operators.py").read_text()
file_lines = file_text.splitlines(True)

prefix = file_lines[2 -1][0:11]
suffix = file_lines[2 -1][16:]
mod = 'x - y'

file_lines[2-1] = prefix + mod + suffix
for _ in range(2, 3):
    file_lines.pop(2)
# file_lines.pop(2)

pprint(file_lines[0:5])
print("".join(file_lines)[:200])

In [ ]:
!set PYTHONPATH=%CD%/src

import ast
from pathlib import Path
from pprint import pprint
import importlib

from poodle import mutators 

importlib.reload(mutators)

file = Path("example/src/operators.py")
parsed = ast.parse(file.read_bytes(), file)

pprint(ast.dump(parsed, indent=4))

bom = mutators.BinaryOperationMutator(None)
# pprint(bom.visit_BinOp(ast.BinOp(left=ast.Constant(4),op=ast.Add(),right=ast.Constant(5),lineno=1,col_offset=1,end_lineno=1,end_col_offset=1,)))
pprint(bom.create_mutants(parsed))